# Testing PatentData on US Grants

This notebook looks at testing the functionality of the patentdata module on granted US patents.

In [1]:
from patentdata.corpus import USGrants, USPublications

c = USGrants("/media/SAMSUNG1/US_Grants")

In [2]:
path = '/patentdata/tests/test_files/Grant'
corpus = USGrants(path)

corpus.first_level_files

['2011/ipg110726_reduced.zip']

In [3]:
corpus.index()

Getting archive file list - may take a while!

Generating list for: 2011
Processing file: 2011/ipg110726_reduced.zip


In [4]:
import patentdata.utils as utils
import os

subdirectory = utils.get_immediate_subdirectories(corpus.path)
subdirectory

['2011']

In [5]:
filtered_files = [
                f for f in corpus.first_level_files
                if subdirectory[0] in os.path.split(f) and "SUPP" not in f
            ]
filtered_files[0]

'2011/ipg110726_reduced.zip'

In [6]:
'2011' in os.path.split(corpus.first_level_files[0])

True

In [7]:
sl, el, xml_doc = next(corpus.read_archive_file(filtered_files[0]))

In [8]:
print(sl, el)

0 1281


In [9]:
pub_details = xml_doc.publication_details()
print(pub_details)

{'short_number': '07984558', 'full_number': 'US07984558B2', 'kind': 'B2', 'date': datetime.datetime(2011, 7, 26, 0, 0)}


In [10]:
records = corpus.c.execute("SELECT * FROM files").fetchall()
records

[('US07984558B2',
  'US',
  2011,
  7984558,
  'B2',
  '2011/ipg110726_reduced.zip',
  None,
  0,
  'G',
  '01',
  'B',
  '5',
  '008'),
 ('US07984559B2',
  'US',
  2011,
  7984559,
  'B2',
  '2011/ipg110726_reduced.zip',
  None,
  1282,
  'G',
  '01',
  'B',
  '1',
  '00')]

In [11]:
records = corpus.get_records(["G"], "start_offset", None)

2 records located.


In [12]:
query_string = utils.build_classification_query(
            ["G"],
            "start_offset"
            )
print(query_string)


                        SELECT ROWID, filename, start_offset
                        FROM files
                        WHERE section = 'G' 
                        


In [13]:
query_string = """SELECT ROWID, filename, start_offset FROM files"""

In [14]:
corpus.c

In [15]:
records = corpus.c.execute(query_string).fetchall()
no_of_records = len(records)
print("{0} records located.".format(no_of_records))

2 records located.


In [16]:
records

[(1, '2011/ipg110726_reduced.zip', 0), (2, '2011/ipg110726_reduced.zip', 1282)]

In [17]:
filegenerator = corpus.iter_filter_xml(["G"])

In [18]:
records = corpus.get_records(["G"], "start_offset", None)
filegenerator = corpus.iter_read(records)

2 records located.


In [19]:
records

[(1, '2011/ipg110726_reduced.zip', 0), (2, '2011/ipg110726_reduced.zip', 1282)]

In [20]:
filelist = records

filename_groups = utils.group_filenames(filelist)
print(filename_groups)

{'2011/ipg110726_reduced.zip': [(1, 0), (2, 1282)]}


In [21]:
# For each filename group
for filename in filename_groups.keys():
    # Get set of second level files
    entries = filename_groups[filename]
    print(entries)
    print([o for _, o in entries])
    for pub_id, filedata in corpus.filedata_generator(
                                                    filename,
                                                    entries
                                                    ):
        print(pub_id)
        #yield pub_id, filedata

[(1, 0), (2, 1282)]
[0, 1282]
None
None


In [22]:
import zipfile
filename = list(filename_groups.keys())[0]
entries = filename_groups[filename]
offset_list = [o for _, o in entries]
print(offset_list)
with zipfile.ZipFile(os.path.join(corpus.path, filename), 'r') as zip_file:
    xml_file = zip_file.namelist()[0]
    print(xml_file)
    offset_list.sort()
    # Reverse list so we can pop from end
    offset_list.reverse()
    print(offset_list)
    

[0, 1282]
ipg110726_reduced.xml
[1282, 0]


In [23]:
start_offset = sorted_offsets.pop()
    with zip_file.open(xml_file, 'r') as open_xml_file:

        for line_no, line in enumerate(open_xml_file):
            if line_no < start_offset:
                continue
            elif line_no == start_offset:
                data_buffer = [line]
            elif line_no >= start_offset:
                if line.startswith(b'<?xml '):
                    # Get next offset if offsets
                    if sorted_offsets:
                        start_offset = sorted_offsets.pop()
                    yield b''.join(data_buffer)
                else:
                    data_buffer.append(line)
        yield b''.join(data_buffer)
    
    for filedata in get_multiple_xml_by_offset(z, offsets):
        yield None, filedata

IndentationError: unexpected indent (<ipython-input-23-a7daa4d78a9f>, line 2)

In [ ]:
print(testlist.pop())

In [ ]:
print(testlist.pop())

In [ ]:
print(testlist.pop())

In [ ]:
records[0][0]

In [ ]:
doc_generator = corpus.patentdoc_generator()
doc = next(doc_generator)
print(doc)